### Task 3

In [1]:
import pandas as pd
import yaml
import logging
import pandas as pd
import os
import sys
import mlflow

sys.path.append(os.path.abspath('../scripts'))

from forecasting.forecast import load_model_from_mlflow, forecast_arima_sarima, forecast_lstm
from forecasting.plot import plot_forecast
from forecasting.analyze import analyze_trend, analyze_volatility

In [2]:
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [3]:
datasets = {
    "TSLA": pd.read_csv("../data/processed/TSLA_processed.csv", index_col="Date", parse_dates=True),
    "SPY": pd.read_csv("../data/processed/SPY_processed.csv", index_col="Date", parse_dates=True),
    "BND": pd.read_csv("../data/processed/BND_processed.csv", index_col="Date", parse_dates=True),
}